# Season data preparation (v2)

This is the second version of the software, which was created to support larger amounts of data. Here, the alignment was performed on all of the episodes from the thirteenth season of *Grey's Anatomy*. The input is the following:

 
1.   An XLSX file containing the data from episode 13x01 to 13x24.
2.   A ZIP file containing the subtitles of each episode.


# Libraries

In addition to pandas and re, some additional libraries were used in order to extract and organize the files.

In [1]:
import pandas as pd
import numpy as np
import tempfile
import zipfile
import shutil
import glob
import uuid
import os
import re

In [2]:
pd.options.mode.chained_assignment = None 

# Importing the files

In [3]:
# Opening season 13 .xlsx file 

excel_path = input('Enter .xlsx file path: ')
df_excel = pd.read_excel(excel_path)

Enter .xlsx file path: /content/GAS01.xlsx


In [4]:
# Structure of the .xlsx file

df_excel

,Series,Season,Codice,Inizio,Fine,PP,SP,MC
0,GA,GAS01,GAS01E01,00:00:00,00:00:17,0,0,0
1,GA,GAS01,GAS01E01,00:00:17,00:01:47,0,6,0
2,GA,GAS01,GAS01E01,00:01:47,00:02:07,0,0,0
3,GA,GAS01,GAS01E01,00:02:07,00:02:52,6,0,0
4,GA,GAS01,GAS01E01,00:02:52,00:02:56,0,0,0
...,...,...,...,...,...,...,...,...
721,GA,GAS01,GAS01E09,00:39:23,00:39:40,0,3,3
722,GA,GAS01,GAS01E09,00:39:40,00:40:19,0,6,0
723,GA,GAS01,GAS01E09,00:40:19,00:40:34,0,6,0
724,GA,GAS01,GAS01E09,00:40:34,00:41:11,0,6,0


In [4]:
# Unzipping the contents of the .zip file to a temporary directory

temp_dir = tempfile.TemporaryDirectory()
zip_path = input('Enter .zip file path: ')

with zipfile.ZipFile(zip_path) as z:
  
   for zip_info in z.infolist():
    if zip_info.filename[-1] == '/':
      continue

    zip_info.filename = os.path.basename(zip_info.filename)
    z.extract(zip_info, temp_dir.name)

print('Temp directory path: ' + str(temp_dir.name))

Enter .zip file path: /content/subtitles_season_1.zip
Temp directory path: /tmp/tmpvt0theo6


In [5]:
# Getting the paths of the extracted .srt files

srt_paths = []

for filename in sorted(glob.glob(os.path.join(temp_dir.name, '*.srt'))):
  srt_paths.append(filename)

# Cleaning the data

Before aligning the files, the following cleaning was performed on the Excel file:
*   Timestamps containing milliseconds were rounded to the nearest second
*   Typographical errors were fixed (e.g. "NA ", "000:09:20,487")
*   NaNs were replaced with 0s
*   Floats were converted to integers





In [6]:
import uuid

# generating unique ids
df_excel['unique_id'] = [str(uuid.uuid4()) for _ in range(len(df_excel))]

# converting them to strings
df_excel['unique_id'] = df_excel['unique_id'].apply(str)

In [ ]:
# Dropping columns

# df_excel = df_excel.drop(['Season', 'N_segmento', 'Durata', 'PP_rel', 'SP_rel', 'MC_rel', 'Note'], axis=1)

In [7]:
# Rounding start timestamps

df_excel['Inizio'] = df_excel['Inizio'].astype(str)
df_excel['Inizio'] = pd.to_datetime(df_excel['Inizio'], errors = 'coerce')
df_excel['Inizio'] = df_excel['Inizio'].dt.round('1s')
df_excel['Inizio'] = df_excel['Inizio'].dt.time

In [8]:
# Rounding end timestamps

df_excel['Fine'] = df_excel['Fine'].astype(str)
df_excel['Fine'] = df_excel['Fine'].apply(lambda x : '00:09:20' if x=='000:09:20,487' else x) # Fixing a value in line 614
df_excel['Fine'] = pd.to_datetime(df_excel['Fine'], errors = 'coerce')
df_excel['Fine'] = df_excel['Fine'].dt.round('1s')
df_excel['Fine'] = df_excel['Fine'].dt.time

In [9]:
# Fixing other typographical errors

df_excel.PP.replace('NA ', np.nan, inplace=True)
df_excel.SP.replace('NA ', np.nan, inplace=True)
df_excel.MC.replace('NA ', np.nan, inplace=True)
df_excel.MC.replace('SP10', np.nan, inplace=True) # Fixing a value in line 786

In [10]:
# Replacing NaNs with 0s

df_excel = df_excel.fillna(0)

In [11]:
# Converting floats to ints

df_excel['PP'] = df_excel['PP'].astype(int)
df_excel['SP'] = df_excel['SP'].astype(int)
df_excel['MC'] = df_excel['MC'].astype(int)

In [13]:
# Structure of the .xlsx file after cleaning

df_excel

,Series,Season,Codice,Inizio,Fine,PP,SP,MC,unique_id
0,GA,GAS01,GAS01E01,00:00:00,00:00:17,0,0,0,79eedf3b-7051-410f-959d-aab268bf4b5d
1,GA,GAS01,GAS01E01,00:00:17,00:01:47,0,6,0,c8f9994d-68ec-4bb8-9574-e3c532d962d0
2,GA,GAS01,GAS01E01,00:01:47,00:02:07,0,0,0,be002fea-df1d-475c-bbed-aa62c2e1afbd
3,GA,GAS01,GAS01E01,00:02:07,00:02:52,6,0,0,916bd242-b8bc-4b94-a41b-02b7bcbffffb
4,GA,GAS01,GAS01E01,00:02:52,00:02:56,0,0,0,c9eeb8af-ee01-45af-9284-881bca8d488f
...,...,...,...,...,...,...,...,...,...
721,GA,GAS01,GAS01E09,00:39:23,00:39:40,0,3,3,7af0b5e9-3df7-44a7-83f3-e6e182dea640
722,GA,GAS01,GAS01E09,00:39:40,00:40:19,0,6,0,2f7bb38c-8f45-47a8-bbf5-69c16631bec7
723,GA,GAS01,GAS01E09,00:40:19,00:40:34,0,6,0,fbec672a-fe8a-47c3-a30e-3253a7388e15
724,GA,GAS01,GAS01E09,00:40:34,00:41:11,0,6,0,9c00d5fb-d406-4ddd-a99a-64a08edfe22b


# Splitting the Excel file by episode

The idea behind the following section is to split the Excel file so that each episode is stored as a separate DataFrame. This way, the procedure followed in episode_12x01_data_preparation.ipynb can be reimplemented for multiple episodes as well.

In [12]:
# Finding where each episode starts and ends

episode_start = []
episode_end = []
episode_start.append(0)

for idx, val in enumerate(df_excel.Codice):

    try:
      if df_excel.Codice[idx] != df_excel.Codice[idx+1]:
        episode_end.append(idx)
        episode_start.append(idx+1)
    
    except Exception:
      episode_end.append(len(df_excel.Codice))

In [13]:
# Boundaries of each episode

print(episode_start)
print(episode_end)

[0, 78, 161, 232, 318, 407, 495, 572, 646]
[77, 160, 231, 317, 406, 494, 571, 645, 726]


In [13]:
# Storing each episode as a separate DataFrame

df_episodes_list = []

for ep_start, ep_end in zip(episode_start, episode_end):

  df_episode = df_excel.loc[ep_start:ep_end]
  df_episodes_list.append(df_episode)

print('Total episodes: ' + str(len(df_episodes_list)))

Total episodes: 9


# Defining the functions

The following functions reimplement the procedure that was described in episode_12x01_data_preparation.ipynb. 

In [14]:
# Computing the mean of each subtitle's timespan, which is used to align the subs with the Excel segments

def average_time(average_times, start_times_datetime, end_times_datetime):

  for s, e in zip(start_times_datetime, end_times_datetime):
    ts1 = s
    ts2 = e
    average_time = ts1+(ts2-ts1)/2
    average_times.append(average_time)

The data extracted from the subtitles and the Excel file can be passed to the functions align_segments() or align_subs(). When executing the code, the user is asked to choose whether to align the data by segment (=to group subtitles that are part of the same segment) or by subtitle (=to display each subtitle independently).

In [15]:
def align_segments(df_subs, codice, inizio, fine, pp, sp, mc, aligned_subs, unique_id):
    
    for code, start, end, pp, sp, mc, id in zip(codice, inizio, fine, pp, sp, mc, unique_id):
        
        mask = (df_subs['Average'] > start) & (df_subs['Average'] <= end)
        mask_df = df_subs.loc[mask]

        mask_df = mask_df.drop(['Average'], axis=1)
        mask_df['PP'] = pp
        mask_df = mask_df.groupby('PP')
        mask_df = mask_df['Subtitle text'].agg(lambda column: ''.join(column))
        

        mask_df = mask_df.reset_index(name='Subtitle text')
        mask_df.rename(columns={'Subtitle text':'Segment text'}, inplace=True)
        mask_df['SP'] = sp
        mask_df['MC'] = mc
        mask_df['Segment start'] = start   
        mask_df['Segment end'] = end
        mask_df['Code'] = code
        mask_df['Segment ID'] = id
        mask_df = mask_df[['Code', 'Segment ID', 'Segment start', 'Segment end', 'PP', 'SP', 'MC', 'Segment text']]
        aligned_subs.append(mask_df)

In [16]:
# If the user selects option [2] when alignment_type() is called, the data is aligned by subtitle

def align_subs(df_subs, codice, inizio, fine, pp, sp, mc, aligned_subs, unique_id):

  for code, start, end, pp, sp, mc, id in zip(codice, inizio, fine, pp, sp, mc, unique_id):
    
    mask = (df_subs['Average'] > start) & (df_subs['Average'] <= end)
    mask_df = df_subs.loc[mask]

    mask_df['PP'] = pp
    mask_df['SP'] = sp
    mask_df['MC'] = mc
    mask_df['Subtitle start'] = mask_df['Start'].astype(str)
    mask_df['Subtitle start'] = mask_df['Subtitle start'].str.replace(r'000$', '', regex=True)  
    mask_df['Subtitle end'] = mask_df['End'].astype(str)
    mask_df['Subtitle end'] = mask_df['Subtitle end'].str.replace(r'000$', '', regex=True) 
    mask_df['Start'] = start   
    mask_df['End'] = end
    mask_df['Code'] = code
    mask_df['Segment ID'] = id
    mask_df = mask_df[['Code', 'Segment ID', 'Subtitle start', 'Subtitle end', 'PP', 'SP', 'MC', 'Subtitle text']]
    aligned_subs.append(mask_df)

For each episode, the process is the same as before:

1.   Parsing and storing subtitles in a DataFrame
2.   Calculating the average of the subtitles' timespans
3.   Using the average to perform the alignment either by segment or by subtitle
4.   Storing the episode's DataFrame in a list 
5.   Repeating the process until all of the episodes have been aligned

In [28]:
import codecs

def process_files(xlsx_file, srt_file, alignment_type, aligned_episodes):

  df_excel = xlsx_file

  with codecs.open(srt_file, 'r', 'utf-8', 'replace') as f:
    subs = f.read().splitlines()
    subs = [x.encode('utf-8', 'replace').decode('utf-8') for x in subs]

    # Parsing .srt file

    re_pattern = r'[0-9]{2}:[0-9]{2}:[0-9]{2},[0-9]{3} -->'
    regex = re.compile(re_pattern)

    start_times = list(filter(regex.search, subs))
    start_times = [time.split(' ')[0] for time in start_times]
    start_times_datetime1 = pd.to_datetime(start_times, format='%H:%M:%S,%f')
    start_times_datetime2 = pd.Series(start_times_datetime1, name='Start').dt.time

    end_times = list(filter(regex.search,subs))
    end_times = [time.split(' ')[2] for time in end_times]
    end_times_datetime1 = pd.to_datetime(end_times, format='%H:%M:%S,%f')
    end_times_datetime2 = pd.Series(end_times_datetime1, name='End').dt.time

    subtitles = [[]]

    for sub in subs:
        if re.match(re_pattern, sub):
          subtitles[-1].pop()
          subtitles.append([])
        else:
          subtitles[-1].append(sub)

    subtitles = subtitles[1:]
    subtitles = [' '.join(x) for x in subtitles]
    subtitles = pd.Series(subtitles, name='Subtitle text', dtype='string')

    # Averaging timespans

    average_times = []
    average_time(average_times, start_times_datetime1, end_times_datetime1)
    average_times = pd.Series(average_times, name='Average').dt.time

    # Creating .srt DataFrame

    df_subs = pd.concat([start_times_datetime2, end_times_datetime2, average_times, subtitles], axis=1)
    df_subs = df_subs.reset_index(drop=True)

    # Matching subtitles with segments

    aligned_subs = []

    if alignment_type == 'segment':
      align_segments(df_subs, df_excel.Codice, df_excel.Inizio, df_excel.Fine, df_excel.PP, df_excel.SP, df_excel.MC, aligned_subs, df_excel.unique_id)
    
    if alignment_type == 'subtitle':
      align_subs(df_subs, df_excel.Codice, df_excel.Inizio, df_excel.Fine, df_excel.PP, df_excel.SP, df_excel.MC, aligned_subs, df_excel.unique_id)

    else:
      pass

    # Appending aligned episode to episode list

    aligned_file = pd.concat(aligned_subs)
    aligned_file.reset_index(drop=True)
    aligned_episodes.append(aligned_file)

# Aligning the episodes

Calling the functions described above to perform the alignment.

In [29]:
aligned_by_segment = []
aligned_by_subtitle = []

def alignment_type(alignment_type):
  
    print(f'Aligning episodes by {alignment_type}...')

    for xlsx, srt in zip(df_episodes_list, srt_paths):

        if alignment_type == 'segment':
            process_files(xlsx, srt, 'segment', aligned_by_segment)
            
        elif alignment_type == 'subtitle':
            process_files(xlsx, srt, 'subtitle', aligned_by_subtitle)

# Perform alignment by segment
alignment_type('segment')

# Perform alignment by subtitle
alignment_type('subtitle')

Aligning episodes by segment...
Aligning episodes by subtitle...


The list containing each episode's DataFrame is merged with pd.concat() to create a single DataFrame with all of the aligned data.

In [30]:
segment_df = pd.concat(aligned_by_segment)
segment_df = segment_df.reset_index(drop=True)
segment_df

,Code,Segment ID,Segment start,Segment end,PP,SP,MC,Segment text
0,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:00,00:00:17,0,0,0,[Woman] The game: They say a person either has...
1,GAS01E01,c6cbec91-5c41-4935-a458-fe4143c3aba6,00:00:17,00:01:47,0,6,0,"[Grunts] [Man] This is, uh... Humiliating on s..."
2,GAS01E01,cb84dbb3-6f07-4425-aee6-aa1764be4a1b,00:01:47,00:02:07,0,0,0,[Rilo Kiley: Portions For Foxes] [Man] Each of...
3,GAS01E01,69e20198-9e4b-48d6-967a-d855f68a53f6,00:02:07,00:02:52,6,0,0,"wanting in on the game. A month ago, you were ..."
4,GAS01E01,d4a6cf34-0235-42f0-bf79-463216653bad,00:03:01,00:03:55,2,4,0,"OK, Martin, Robinson, Bond, Hawkins. Only six ..."
...,...,...,...,...,...,...,...,...
676,GAS01E09,c89340a4-e3b8-4340-aa60-f833a1832392,00:39:23,00:39:40,0,3,3,If you could just... sign this consent form fo...
677,GAS01E09,4e824540-f4c7-4cbf-a319-6dddf334c684,00:39:40,00:40:19,0,6,0,The thing people forget is how good it can fee...
678,GAS01E09,ab5af7bc-4090-4d93-bb82-fafd61c3c103,00:40:19,00:40:34,0,6,0,"I didn't know how much I'd like you, and when ..."
679,GAS01E09,57d4ac4b-793d-4d81-901b-75a8d29b499f,00:40:34,00:41:11,0,6,0,You and Alex?! You gave me syphilis?! George! ...


In [31]:
subtitle_df = pd.concat(aligned_by_subtitle)
subtitle_df = subtitle_df.reset_index(drop=True)
subtitle_df

,Code,Segment ID,Subtitle start,Subtitle end,PP,SP,MC,Subtitle text
0,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:03.770,00:00:05.635,0,0,0,[Woman] The game:
1,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:05.705,00:00:08.435,0,0,0,They say a person either has what it takes to ...
2,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:08.508,00:00:10.305,0,0,0,or they don't.
3,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:10.377,00:00:13.676,0,0,0,My mother was one of the greats.
4,GAS01E01,3810151b-e2cb-45ec-813b-f3fb63725af7,00:00:13.747,00:00:17.649,0,0,0,"Me, on the other hand... I'm kind of screwed."
...,...,...,...,...,...,...,...,...
6766,GAS01E09,15bfcee4-0b9d-47f8-983a-35727902399b,00:41:38.033,00:41:42.754,0,6,0,The problem with secrets is even when you thin...
6767,GAS01E09,15bfcee4-0b9d-47f8-983a-35727902399b,00:41:43.808,00:41:44.801,0,6,0,Addison.
6768,GAS01E09,15bfcee4-0b9d-47f8-983a-35727902399b,00:41:45.166,00:41:46.123,0,6,0,...You're not.
6769,GAS01E09,15bfcee4-0b9d-47f8-983a-35727902399b,00:41:46.383,00:41:47.750,0,6,0,What are you doing here?


# Deleting temporary files

Deleting the temporary directory where the extracted .srt files were stored.

In [ ]:
shutil.rmtree(temp_dir.name)

# Exporting to Excel

In [36]:
def export_to_excel(segment_df, subtitle_df):
    segment_file_name = segment_df.iloc[0, 0][:5] + '_with_segments.xlsx'
    subtitle_file_name = segment_df.iloc[0, 0][:5] + '_with_subtitles.xlsx'
    segment_df.to_excel(segment_file_name, index=False)
    subtitle_df.to_excel(subtitle_file_name, index=False)
    print(f"Exported to Excel '{segment_file_name}' and '{subtitle_file_name}'")

In [37]:
export_to_excel(segment_df, subtitle_df)

Exported to Excel 'GAS01_with_segments.xlsx' and 'GAS01_with_subtitles.xlsx'
